Aquí profundizaremos en los generadores de Python, incluidas las expresiones del generador y las funciones del generador .

##### Expresiones generadoras 
La diferencia entre las comprensiones de listas y las expresiones generadoras es a veces confusa; aquí describiremos rápidamente las diferencias entre ellos:

Las listas por comprensión usan corchetes, mientras que las expresiones generadoras usan paréntesis 

Esta es una lista de comprensión representativa:

In [5]:
[n ** 2 for n in range(12)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

Si bien esta es una expresión generadora representativa:

In [6]:
(n ** 2 for n in range(12))

<generator object <genexpr> at 0x000001FD121E0120>

Observe que al imprimir la expresión del generador no se imprime el contenido; una forma de imprimir el contenido de una expresión generadora es pasárselo al constructor list:

In [9]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

**Una lista es una colección de valores, mientras que un generador es una receta para producir valores**

***Cuando crea una lista, en realidad está creando una colección de valores, y hay cierto costo de memoria asociado con eso. Cuando crea un generador, no está creando una colección de valores, sino una receta para producir esos valores.*** Ambos exponen la misma interfaz de iterador, como podemos ver aquí:

In [11]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

In [12]:
G = (n ** 2 for n in range(12))
for val in G:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

***La diferencia es que una expresión generadora en realidad no calcula los valores hasta que se necesitan.*** 
¡Esto no solo conduce a la eficiencia de la memoria, sino también a la eficiencia computacional! Esto también significa que, si bien el tamaño de una lista está limitado por la memoria disponible, el tamaño de una expresión generadora es ilimitado.

Se puede crear un ejemplo de una expresión generadora infinita usando el iterador count definido en itertools:

In [15]:
from itertools import count
count()

count(0)

In [16]:
for i in count():
    print(i, end=' ')
    if i >= 10: break

0 1 2 3 4 5 6 7 8 9 10 

El  iterador count seguirá contando felizmente para siempre hasta que le diga que se detenga; esto hace que sea conveniente crear generadores que también funcionarán para siempre:

In [17]:
factors = [2, 3, 5, 7]
G = (i for i in count() if all(i % n > 0 for n in factors))
for val in G:
    print(val, end=' ')
    if val > 40: break

1 11 13 17 19 23 29 31 37 41 

Es posible que vea a lo que estamos llegando aquí: si expandiéramos la lista de factores de manera adecuada, lo que tendríamos al comienzo es un generador de números primos, utilizando el algoritmo Sieve of Eratosthenes. Exploraremos esto más momentáneamente.

***Una lista se puede iterar varias veces; una expresión generadora es de un solo uso***

Esta es una de esas posibles trampas de las expresiones generadoras. Con una lista, podemos hacer esto directamente:

In [23]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')
print()

for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 
0 1 4 9 16 25 36 49 64 81 100 121 

**Una expresión generadora, por otro lado, se agota después de una iteración:**

In [24]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

In [25]:
list(G)

[]

**Esto puede ser muy útil porque significa que la iteración se puede detener e iniciar**

In [26]:
G = (n**2 for n in range(12))
for n in G:
    print(n, end=' ')
    if n > 30: break

print("\ndoing something in between")

for n in G:
    print(n, end=' ')

0 1 4 9 16 25 36 
doing something in between
49 64 81 100 121 

**Un lugar en el que he encontrado esto útil es cuando trabajo con colecciones de archivos de datos en el disco; significa que puede analizarlos fácilmente en lotes, dejando que el generador realice un seguimiento de los que aún no ha visto.**

#### Funciones del generador: usando yield

**Vimos en la sección anterior que las listas por comprensión se usan mejor para crear listas relativamente simples, mientras que usar un ciclo for normal puede ser mejor en situaciones más complicadas.**

Lo mismo ocurre con las expresiones generadoras: podemos hacer generadores más complicados usando funciones generadoras , que hacen uso de la declaración **yield**.

Aquí tenemos dos formas de construir la misma lista:

In [31]:
L1 = [n ** 2 for n in range(12)]

L2 = []
for n in range(12):
    L2.append(n ** 2)

print(L1)
print(L2)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]


De manera similar, aquí tenemos dos formas de construir generadores equivalentes:

In [32]:
G1 = (n ** 2 for n in range(12))

def gen():
    for n in range(12):
        yield n ** 2

G2 = gen()
print(*G1)
print(*G2)

0 1 4 9 16 25 36 49 64 81 100 121
0 1 4 9 16 25 36 49 64 81 100 121


**Una función generadora es una función que, en lugar de usar return para devolver un valor una vez, usa yield para producir una secuencia de valores (potencialmente infinita).**

**Al igual que en las expresiones del generador, el estado del generador se conserva entre iteraciones parciales, pero si queremos una copia nueva del generador, simplemente podemos volver a llamar a la función.**

#### Ejemplo: generador de números primos 
Aquí mostraré mi ejemplo favorito de una función generadora: una función para generar una serie ilimitada de números primos. Un algoritmo clásico para esto es el Tamiz de Eratóstenes , que funciona así:

In [34]:
# Generate a list of candidates
L = [n for n in range(2, 40)]
print(L)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [35]:
# Remove all multiples of the first value
L = [n for n in L if n == L[0] or n % L[0] > 0]
print(L)

[2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39]


In [36]:
# Remove all multiples of the second value
L = [n for n in L if n == L[1] or n % L[1] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 25, 29, 31, 35, 37]


In [37]:
# Remove all multiples of the third value
L = [n for n in L if n == L[2] or n % L[2] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]


Si repetimos este procedimiento suficientes veces en una lista lo suficientemente grande, podemos generar tantos números primos como deseemos.

Encapsulemos esta lógica en una función generadora:

In [40]:
def gen_primes(N):
    """Generate primes up to N"""
    primes = set()
    for n in range(2, N):
        if all(n % p > 0 for p in primes):
            primes.add(n)
            yield n

print(*gen_primes(100))

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97


¡Eso es todo al respecto! Si bien esta no es ciertamente la implementación más eficiente desde el punto de vista computacional del Tamiz de Eratóstenes, ilustra lo conveniente que puede ser la sintaxis de la función del generador para construir secuencias más complicadas.